In [7]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import matplotlib.pyplot as plt
from pprint import pprint

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

import warnings
warnings.filterwarnings("ignore")

In [28]:
training_data = pd.read_csv("./DateDatasets/training.csv").drop(columns=["Unnamed: 0"])
testing_data = pd.read_csv("./DateDatasets/testing.csv").drop(columns=["Unnamed: 0"])
validating_data = pd.read_csv("./DateDatasets/validation.csv").drop(columns=["Unnamed: 0"])

In [29]:
cols = ["StationEnd", "Count", "Count1", "Count1week", "Temp", "ATemp", "Hour", "Windspeed"]

training_data = training_data[[col for col in training_data.columns if col in cols]]
testing_data = testing_data[[col for col in testing_data.columns if col in cols]]
validating_data = validating_data[[col for col in validating_data.columns if col in cols]]

In [30]:
trainX = training_data.drop(columns=["Count"])
trainY = training_data["Count"]

testX = testing_data.drop(columns=["Count"])
testY = testing_data["Count"]

valX = validating_data.drop(columns=["Count"])
valY = validating_data["Count"]

trainX = np.reshape(trainX.values, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX.values, (testX.shape[0], 1, testX.shape[1]))
valX = np.reshape(valX.values, (valX.shape[0], 1, valX.shape[1]))

In [34]:
model = Sequential()
model.add(LSTM(4, input_shape=(1,training_data.shape[1]-1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=40, batch_size=1, verbose=2)

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(pd.concat([training_data, testing_data]))

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

Epoch 1/40


2023-05-06 19:31:36.566151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-06 19:31:36.567229: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-06 19:31:36.567858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [31]:
trainPredict = model.predict(trainX)

trainScore = np.sqrt(mean_squared_error(trainY[:], trainPredict[:]))
print('Train Score: %.2f RMSE' % (trainScore))
results = model.evaluate(trainX, trainY, batch_size=128)
print("Train Accuracy:", results)

4593/4593 [==============================] - 2s 526us/step
Train Score: 3.18 RMSE
1149/1149 [==============================] - 1s 584us/step - loss: 10.0972
Train Accuracy: 10.097208976745605


In [32]:
testPredict = model.predict(testX)

testScore = np.sqrt(mean_squared_error(testY[:], testPredict[:]))
print('Test Score: %.2f RMSE' % (testScore))
results = model.evaluate(testX, testY, batch_size=128)
print("Test Accuracy:", results)

1286/1286 [==============================] - 1s 556us/step
Test Score: 3.76 RMSE
322/322 [==============================] - 0s 608us/step - loss: 14.1328
Test Accuracy: 14.13275146484375


In [33]:
valPredict = model.predict(valX)

valScore = np.sqrt(mean_squared_error(valY[:], valPredict[:]))
print('Validation Score: %.2f RMSE' % (valScore))
results = model.evaluate(valX, valY, batch_size=128)
print("Validation Accuracy:", results)

1189/1189 [==============================] - 1s 589us/step
Validation Score: 3.41 RMSE
298/298 [==============================] - 0s 581us/step - loss: 11.6192
Validation Accuracy: 11.61919116973877
